In [ ]:
# Instalação de dependências
!pip install unidecode # remove acentos de strings

# Chatbots

Neste tutorial, vamos criar um chatbot utilizando expressões regulares. Existem várias técnicas para criar chatbots e, por mais que expressões regulares não sejam o estado da arte em chatbots, é possível se divertir e criar alguns chatbots razoáveis.

## O que é um chatbot?

Abstraindo ao máximo, podemos dizer que um chatbot é uma função que recebe um contexto e uma entrada do usuário e retorna um novo contexto e uma string com a resposta.

In [ ]:
def yell_bot(ctx, msg):
    """
    Bot que responde gritando de volta ao usuário.
    """
    return ctx, msg.upper() + '!!!'

O contexto pode ser qualquer coisa que represente o atual contexto de conversa. Desde uma string até representações mais complicadas que seu chatbot possa utilizar. A única coisa que importa é que o chatbot entenda o contexto recebido e sempre retorne contextos que ele consiga processar. 

Nosso `yell_bot` é extremamente simples. Mas um chatbot extremamente complexo é capaz de operar utilizando a mesma interface. Interagimos com o bot passando-o como argumento para a função interact.

In [ ]:
from time import sleep
from unidecode import unidecode  


def interact(chatbot, ctx=None, intro=None):
    """Chama o chatbot em loop passando o contexto inicial."""

    # Mensagem opcional no início
    if intro is not None:
        respond(intro)
        
    while True:
        try:
            msg = input('> ')
        except KeyboardInterrupt:
            print()
            msg = ''

        if not msg:
            if input('sair? (S/n) ').lower() in 's':
                break
        
        ctx, reply = chatbot(ctx, clean(msg))
        respond(reply)

        
def respond(msg):
    for c in msg:
        print(c, end='')
        if c == '\n':
            sleep(0.2)
        sleep(0.05)
    print('\n')
    sleep(0.4)
        
def clean(x):
    return unidecode(x.lower())
        

interact(yell_bot)

Vamos criar uma função que ajuda em testes e na execução automática do nosso bot.

In [ ]:
def replay(msgs, chatbot, ctx=None, intro=None):
    """
    Executa interação de forma não-interativa. Útil para testes.
    """
    if intro is not None:
        respond(intro)
        
    for msg in msgs:
        print('>', msg)
        ctx, reply = chatbot(ctx, clean(msg))
        respond(reply)


replay(['oi, tudo bem?', 'eu estou!'], yell_bot)

## Funcbot

Agora que temos uma interface bem definida, podemos criar funções mais sofisticadas que o `yell_bot` para criar nossos chatbots. É lógico que é muito difícil fazer isto sem uma estrutura, ainda mais que conversas com seres humanos tendem a ser altamente contextuais e dependentes de informação externa.

Vamos criar uma estrutura simples para compor nossos chatbots. A idéia básica é criar chatbots mais complicados a partir de chatbots mais simples. Como exemplo, considere a função compose abaixo. Ela cria um chatbot que retorna o resultado da interação do primeiro chatbot que compreendeu a mensagem e não retornou uma string vazia.

In [ ]:
def compose(*bots):
    """Cria chatbot que retorna a primeira interação bem sucedida 
    na lista de bots fornecidas."""
    
    def bot(ctx, msg):
        for bot in bots:
            ctx_, msg_ = bot(ctx, msg)
            if msg_:
                return ctx_, msg_
        return ctx, ''
    
    return bot

Também podemos criar algumas funções úteis para criar bots ultra especializados

In [ ]:
from random import choice


def eq(expected, reply, to=None):
    """
    Retorna reply se entrada do usuário for exatamente igual ao valor 
    de msg.
    """
    
    expected = expected.lower()
    
    def eq_bot(ctx, msg):
        if msg.lower() == expected:
            return output(ctx, to, reply)
        return ctx, ''
    
    return eq_bot


def has(words, reply, to=None):
    """
    Retorna reply se entrada do usuário contiver
    alguma das palavras em words.
    """
    
    words = [w.lower() for w in words]
    def has_bot(ctx, msg):
        msg = msg.lower()
        if any(w in msg for w in words):
            return output(ctx, to, reply)
        return ctx, ''
    
    return has_bot


def always(reply, to=None):
    """
    Sempre retorna a mesma mensagem.
    """
    
    def bot(ctx, msg):
        return output(ctx, to, reply)
    return bot


def neverbot(msg, ctx):
    """
    Bot que nunca é selecionado
    """
    return ctx, ''


# Normaliza saídas.
def output(curr_ctx, ctx, reply, data=None):
    """
    Normaliza o padrão de saída:
    
    * Se ctx for None, escolhe curr_ctx. 
    * Se reply for uma função, utiliza reply()
    * Se for conjunto, lista ou tupla, sorteia elemento aleatorio
    * Se data for dado, formata string com dicionário de
      valores.
    """
    
    ctx = curr_ctx if ctx is None else ctx
    if callable(reply):
        reply = reply()
    elif isinstance(reply, (tuple, list, set)):
        reply = choice(list(reply))
    if data is not None:
        reply = reply.format(**data)
    return ctx, reply

Também criamos formas mais sofisticadas de composição de bots.

In [ ]:
def if_msg(cond, bot, else_=neverbot):
    """
    Executa bot somente se função fornecida for avaliada como True na
    entrada do usuário.
    """
    def if_bot(ctx, msg):
        if cond(msg):
            return bot(ctx, msg)
        return else_(ctx, msg)
    return bot


def if_ctx(cond, bot, else_=neverbot):
    """
    Executa bot somente se função fornecida for avaliada como True no
    contexto atual.
    """
    def if_bot(ctx, msg):
        if cond(ctx):
            return bot(ctx, msg)
        return else_(ctx, msg)
    return bot


def story(name, *bots):
    """
    Cria uma história linear.
    
    Nenhuma parte da história pode ser capturada por uma
    regra anterior.
    """
    
    def bot(ctx, msg):
        if ctx and ctx.startswith(name + ':'):
            idx = int(ctx.partition(':')[-1])
            _, reply = bots[idx](ctx, msg)
            
            if reply:
                if idx == len(bots) - 1:
                    return None, reply
                return f'{name}:{idx + 1}', reply
            return None, ''
        else:
            ctx, reply = bots[0](ctx, msg)
            if reply:
                return f'{name}:1', reply
            return ctx, ''
        
    return bot


# Funções auxiliares para utilizar em if_ctx, if_msg
def one_of(*opts):
    return lambda x: x in opts

def has_one_of(*opts):
    return lambda x: any(a in x for x in opts)

def has_all_of(*opts):
    return lambda x: all(a in x for x in opts)

# Crie outras funções úteis aqui...

Temos agora um arsenal para criar pequenos bots:

In [ ]:
simple_bot = compose(
    story('say-hello', 
        has(['oi', 'ola'], 'Olá! Tudo bem?'),
        if_msg(has_one_of('sim', 'bem', 'ok'), 
            always('Eu também :)'),
            always('Hmm... Qual é o seu nome?'),
        ),
        always('Mudando o assunto, você gosta de compiladores?'),
    ),
)

replay(['Olá!', 'Sim, e você?', 'Que bom...', 'Sim'], simple_bot)

## Expressões regulares

Agora você assume. Queremos  enriquecer nosso bot adicionando algumas funções baseadas em expressões regulares. Expressões regulares permitem criar padrões mais sofisticados e passar valores para a resposta do bot.

In [ ]:
import re

def regex(pattern, reply, to=None):
    """
    Função captura padrão e passa o dicionário de valores 
    para o output
    """
    search = re.compile(pattern).search
    
    def match_bot(ctx, msg):
        m = search(msg)
        if m:
            return output(ctx, to, reply, m.groupdict())
        else:
            return ctx, ''
        
    return match_bot

Com o match bot, podemos fazer bots que extraiam informação da mensagem do usuário.

In [ ]:
bot = regex(r'tenho (?P<anos>\d+) anos\.?', 
            'Anotado: {anos} anos...\nAgora diga seu e-mail.')
replay(['tenho 18 anos'], bot, intro='Quantos anos você tem?')

Crie um bot que peça o endereço de e-mail de uma pessoa.

In [ ]:
mail_bot = story('ask-email', 
    regex(r'(?P<email>NAO-FUNCIONA)', 'Seu e-mail é {email}. Confirma?'),
    if_msg(has_one_of('sim', 'ok'),
        always('Obrigado.'),
        always('Você pode repetir?', to='ask-email'),
    ),
)

replay(['Meu e-mail é foo@bar.com.', 'Sim'], mail_bot)

Agora faça um bot que consiga manter uma conversa genérica por algumas interações.

In [ ]:
# Tente programar/testar cada caso separadamente. Crie vários bots e componha.
gama = compose(
    has(['poeira'], 'É brabo mesmo.'),
    has(['almoçar'], 'Vou almoçar no RU mesmo.'),
)

nome = regex('meu nome eh? (?P<nome>\w+)', 'Prazer, {nome}!')
erro = always(['Desculpe, não entendi.', 'Pode repetir?'])

# No final, junte todos os sub-bots.
bot = compose(
    nome,
    gama,
    erro,
)
replay(['Oi tudo bem?', 
        'Meu nome é Test-o'], bot)

### Dica

O bot parecerá mais autêntico se estiver em um contexto em que é aceitável dar respostas confusas e vagas ou puder recorrer a bordões e afirmações genéricas. A abordagem que mostrei aqui precisa evoluir muito até conseguir fazer um bot que seja útil. No entanto, se você escolher um domínio adequado, é possível enganar algumas pessoas num teste de Turing.

Talvez por isto o primeiro bot criado tenha sido um psicoterapeuta. Outras boas opções são guru espiritual, astrólogo, político, "life coach", juiz, presidente com baixo nível intelectual, filósofo pós-modernista, crítico de arte, etc.